In [39]:
import requests
import json

# 추후 혼잡도 데이터 업데이트를 위한 INFO parsing
INFO_URL = 'https://infuser.odcloud.kr/oas/docs?namespace=15071311/v1'
API_KEY = '7x1TU10Ct0763mZVXpBvYSYizKLENWLSWK9K1J6pz0euL7mh7iZhN0TEKp48968l7TFlRh9SXPsO%2BANWRXaDUA%3D%3D'

path_raw = requests.get(INFO_URL)
path_data = json.loads(path_raw.text)

update_path = list(path_data['paths'])[-1] # update 시 이코드 사용

# 지하철 혼잡도 API
CONGESTION_API = f'https://api.odcloud.kr/api{update_path}?page=1&perPage=2000&serviceKey={API_KEY}'

cong_raw = requests.get(CONGESTION_API)
cong_data = json.loads(cong_raw.text)

# 지하철 경로 검색 API
dept = '2728' # 출발지 역코드
dest = '0214' # 도착지 역코드
week = 'DAY' # 주중(DAY), 토요일(SAT), 공휴일(HOL)
search_type = 'FASTEST' # (최소 시간)FASTEST, (최소 환승)MINTRF
dept_time = '120001' # (출발 시간)HHmmss

SUBWAY_NAV_API = f'https://apis.data.go.kr/B553766/smt-path/path?serviceKey={API_KEY}&pageNo=1&numOfRows=300&dept_station_code={dept}&dest_station_code={dest}&week={week}&search_type={search_type}&dept_time={dept_time}'

nav_raw = requests.get(SUBWAY_NAV_API)
nav_data = json.loads(nav_raw.text)

In [27]:
list(path_data['paths'])[-3:]

['/15071311/v1/uddi:a5158b81-27c7-4151-ba6c-b912a6f13d39',
 '/15071311/v1/uddi:70e3a3d3-0872-4828-8234-f0bca459b44f',
 '/15071311/v1/uddi:b3803d43-ffe3-4d17-9024-fd6cfa37c284']

In [69]:
# API 끼리 역코드가 맞는지 확인
search = [int(dept), int(dest)]

for i in range(cong_data['currentCount']):
  if cong_data['data'][i]['역번호'] in search:
    print(cong_data['data'][i]['역명'], cong_data['data'][i]['역번호'])

강변 214
강변 214
어린이대공원 2728
어린이대공원 2728
강변 214
강변 214
어린이대공원 2728
어린이대공원 2728
강변 214
강변 214
어린이대공원 2728
어린이대공원 2728


In [150]:
# 상선 / 하선, 내선 / 외선 넘버링 확인
for i in cong_data['data']:
  if i['호선'] == 1:
    print(i['역명'], i['역번호'], i['구분'], i['10시00분'])

서울역 150 상선 26.6
서울역 150 하선 13.9
시청 151 상선 20.2
시청 151 하선 13.5
종각 152 상선 17.0
종각 152 하선 14.9
종로3가 153 상선 19.7
종로3가 153 하선 17.5
종로5가 154 상선 18.7
종로5가 154 하선 20.9
동대문 155 상선 17.2
동대문 155 하선 22.9
신설동 156 상선 13.7
신설동 156 하선 25.6
제기동 157 상선 10.5
제기동 157 하선 26.9
청량리 158 상선 7.3
청량리 158 하선 25.7
동묘앞 159 상선 15.0
동묘앞 159 하선 23.7
서울역 150 상선 28.5
서울역 150 하선 21.2
시청 151 상선 21.9
시청 151 하선 19.6
종각 152 상선 19.9
종각 152 하선 21.7
종로3가 153 상선 23.5
종로3가 153 하선 21.3
종로5가 154 상선 20.0
종로5가 154 하선 25.4
동대문 155 상선 18.5
동대문 155 하선 26.4
신설동 156 상선 16.9
신설동 156 하선 27.8
제기동 157 상선 13.4
제기동 157 하선 27.4
청량리 158 상선 10.6
청량리 158 하선 28.7
동묘앞 159 상선 17.5
동묘앞 159 하선 28.8
서울역 150 상선 21.2
서울역 150 하선 15.4
시청 151 상선 18.8
시청 151 하선 14.1
종각 152 상선 17.2
종각 152 하선 15.2
종로3가 153 상선 18.6
종로3가 153 하선 15.2
종로5가 154 상선 16.4
종로5가 154 하선 17.6
동대문 155 상선 16.0
동대문 155 하선 18.4
신설동 156 상선 11.6
신설동 156 하선 19.7
제기동 157 상선 9.5
제기동 157 하선 19.5
청량리 158 상선 8.1
청량리 158 하선 21.5
동묘앞 159 상선 13.2
동묘앞 159 하선 19.2


In [148]:
cong_data

{'currentCount': 1704,
 'data': [{'10시00분': '26.6',
   '10시30분': '23.4',
   '11시00분': '22.4',
   '11시30분': '22.1',
   '12시00분': '21.5',
   '12시30분': '27.2',
   '13시00분': '21.2',
   '13시30분': '24.1',
   '14시00분': '21.7',
   '14시30분': '18.7',
   '15시00분': '18.5',
   '15시30분': '18.4',
   '16시00분': '16.4',
   '16시30분': '17.7',
   '17시00분': '16.7',
   '17시30분': '24.1',
   '18시00분': '30.8',
   '18시30분': '28.7',
   '19시00분': '20.0',
   '19시30분': '15.1',
   '20시00분': '11.1',
   '20시30분': '12.0',
   '21시00분': '12.5',
   '21시30분': '10.1',
   '22시00분': '10.4',
   '22시30분': '16.4',
   '23시00분': '7.8',
   '23시30분': '6.1',
   '5시30분': '6.2',
   '6시00분': '14.6',
   '6시30분': '13.4',
   '7시00분': '26.8',
   '7시30분': '43.3',
   '8시00분': '53.6',
   '8시30분': '70.9',
   '9시00분': '40.4',
   '9시30분': '38.0',
   '구분': '상선',
   '역명': '서울역',
   '역번호': 150,
   '연번': 1,
   '조사일자': '평일',
   '호선': 1},
  {'10시00분': '13.9',
   '10시30분': '15.2',
   '11시00분': '14.6',
   '11시30분': '14.2',
   '12시00분': '17.6',
   '12시30분'

In [43]:
nav_data

{'data': {'arrv_time': '121800',
  'start_st': '2728',
  'end_st': '0214',
  'week': 'DAY',
  'transfer': 1,
  'route': [{'station_cd': '2728',
    'station_nm': '어린이대공원',
    'timestamp': '120340',
    'train_code': '7187',
    'line_num': '7',
    'transfer_loc': None},
   {'station_cd': '2729',
    'station_nm': '건대입구',
    'timestamp': '120540',
    'train_code': '7187',
    'line_num': '7',
    'transfer_loc': '8-4'},
   {'station_cd': '0212',
    'station_nm': '건대입구',
    'timestamp': '121400',
    'train_code': '2218',
    'line_num': '2',
    'transfer_loc': None},
   {'station_cd': '0213',
    'station_nm': '구의',
    'timestamp': '121630',
    'train_code': '2218',
    'line_num': '2',
    'transfer_loc': None},
   {'station_cd': '0214',
    'station_nm': '강변',
    'timestamp': '121800',
    'train_code': '2218',
    'line_num': '2',
    'transfer_loc': None}],
  'fee': 1350,
  'dept_time': '120001',
  'time': '1420',
  'search_type': 'FASTEST'},
 'pageNo': '1',
 'currentCount

In [123]:
update_path = list(path_data['paths'])[2] # update 시 이코드 사용

# 지하철 혼잡도 API
CONGESTION_API = f'https://api.odcloud.kr/api{update_path}?page=1&perPage=2000&serviceKey={API_KEY}'

cong_raw = requests.get(CONGESTION_API)
cong_data = json.loads(cong_raw.text)

In [135]:
a, b = list(cong_data['data'][0].items())[:-6][0]

In [133]:
import numpy as np

a = [[1, 2, 3], [4, 5, 6]]

x = np.array([y for y in a])
x

array([[1, 2, 3],
       [4, 5, 6]])

In [147]:
from datetime import date, datetime

datetime(2012, 3, 4, 1, 2, 3) < datetime.now()

True

In [154]:
for data in cong_data['data']:
  for t, c in list(data.items())[:-6]:
    print(t, c)

10시00분 26.6
10시30분 23.4
11시00분 22.4
11시30분 22.1
12시00분 21.5
12시30분 27.2
13시00분 21.2
13시30분 24.1
14시00분 21.7
14시30분 18.7
15시00분 18.5
15시30분 18.4
16시00분 16.4
16시30분 17.7
17시00분 16.7
17시30분 24.1
18시00분 30.8
18시30분 28.7
19시00분 20.0
19시30분 15.1
20시00분 11.1
20시30분 12.0
21시00분 12.5
21시30분 10.1
22시00분 10.4
22시30분 16.4
23시00분 7.8
23시30분 6.1
5시30분 6.2
6시00분 14.6
6시30분 13.4
7시00분 26.8
7시30분 43.3
8시00분 53.6
8시30분 70.9
9시00분 40.4
9시30분 38.0
10시00분 13.9
10시30분 15.2
11시00분 14.6
11시30분 14.2
12시00분 17.6
12시30분 19.6
13시00분 20.3
13시30분 20.5
14시00분 25.4
14시30분 22.0
15시00분 30.1
15시30분 24.4
16시00분 33.8
16시30분 36.7
17시00분 43.8
17시30분 49.7
18시00분 84.0
18시30분 65.1
19시00분 37.5
19시30분 26.5
20시00분 25.5
20시30분 26.4
21시00분 26.3
21시30분 21.1
22시00분 32.1
22시30분 20.3
23시00분 12.6
23시30분 10.3
5시30분 9.1
6시00분 8.6
6시30분 9.3
7시00분 15.2
7시30분 29.3
8시00분 27.7
8시30분 19.0
9시00분 18.5
9시30분 15.1
10시00분 20.2
10시30분 21.9
11시00분 20.8
11시30분 20.0
12시00분 21.7
12시30분 24.4
13시00분 21.7
13시30분 23.7
14시00분 21.5
14시30분 20.7
15시00분 20.3
15시3